## 1. JigsawPuzzle Class:

In [1]:
import random
from PIL import Image

class JigsawPuzzle:
    def __init__(self, image, size):
        self.image = image
        self.size = size
        self.width, self.height = self.image.size
        self.piece_width = self.width // size
        self.piece_height = self.height // size
        self.puzzle = self.create_puzzle()
        self.position = (size - 1, size - 1)
        self.shuffle()

    def create_puzzle(self):
        puzzle = []
        for i in range(self.size):
            row = []
            for j in range(self.size):
                left = j * self.piece_width
                upper = i * self.piece_height
                right = left + self.piece_width
                lower = upper + self.piece_height
                piece = self.image.crop((left, upper, right, lower))
                row.append(piece)
            puzzle.append(row)
        puzzle[self.size - 1][self.size - 1] = Image.new('RGB', (self.piece_width, self.piece_height), (255, 255, 255))
        return puzzle

    def shuffle(self):
        for _ in range(self.size ** 3):
            moves = self.get_moves()
            move = random.choice(moves)
            self.move(move)

    def get_moves(self):
        moves = []
        if self.position[0] > 0:
            moves.append('up')
        if self.position[0] < self.size - 1:
            moves.append('down')
        if self.position[1] > 0:
            moves.append('left')
        if self.position[1] < self.size - 1:
            moves.append('right')
        return moves

    def move(self, direction):
        x, y = self.position
        if direction == 'up':
            self.puzzle[x][y], self.puzzle[x - 1][y] = self.puzzle[x - 1][y], self.puzzle[x][y]
            self.position = (x - 1, y)
        elif direction == 'down':
            self.puzzle[x][y], self.puzzle[x + 1][y] = self.puzzle[x + 1][y], self.puzzle[x][y]
            self.position = (x + 1, y)
        elif direction == 'left':
            self.puzzle[x][y], self.puzzle[x][y - 1] = self.puzzle[x][y - 1], self.puzzle[x][y]
            self.position = (x, y - 1)
        elif direction == 'right':
            self.puzzle[x][y], self.puzzle[x][y + 1] = self.puzzle[x][y + 1], self.puzzle[x][y]
            self.position = (x, y + 1)

    def display(self):
        display_image = Image.new('RGB', (self.width, self.height))
        for i in range(self.size):
            for j in range(self.size):
                display_image.paste(self.puzzle[i][j], (j * self.piece_width, i * self.piece_height))
        display_image.show()

    def is_solved(self):
        return all(self.puzzle[i][j] == self.image.crop((j * self.piece_width, i * self.piece_height, (j + 1) * self.piece_width, (i + 1) * self.piece_height)) for i in range(self.size) for j in range(self.size - 1)) and \
               self.puzzle[self.size - 1][self.size - 1] == Image.new('RGB', (self.piece_width, self.piece_height), (255, 255, 255))


## 2. PuzzleGUI Class:

In [2]:
import tkinter as tk
from PIL import Image, ImageTk
import requests
from io import BytesIO

class PuzzleGUI:
    def __init__(self, root, image, size):
        self.root = root
        self.root.title("Jigsaw Puzzle")
        self.image = image
        self.size = size
        self.width, self.height = self.image.size
        self.piece_width = self.width // size
        self.piece_height = self.height // size
        self.puzzle = JigsawPuzzle(image, size)
        self.tiles = [[None] * size for _ in range(size)]
        self.images = [[None] * size for _ in range(size)]
        self.create_canvas()
        self.create_buttons()
        self.draw()

    def create_canvas(self):
        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height, bd=0, highlightthickness=0)
        self.canvas.pack()
        self.canvas.bind("<Button-1>", self.click)

    def create_buttons(self):
        self.solve_button = tk.Button(self.root, text="Solve", command=self.solve)
        self.solve_button.pack()

    def draw(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.puzzle.puzzle[i][j] != None:
                    piece = ImageTk.PhotoImage(self.puzzle.puzzle[i][j])
                    self.images[i][j] = piece  # Keeping a reference to the image
                    self.tiles[i][j] = piece
                    self.canvas.create_image(j * self.piece_width + self.piece_width // 2,
                                             i * self.piece_height + self.piece_height // 2,
                                             image=piece)

    def click(self, event):
        x, y = event.x, event.y
        col = x // self.piece_width
        row = y // self.piece_height
        if (abs(row - self.puzzle.position[0]) + abs(col - self.puzzle.position[1])) == 1:
            self.puzzle.move(self.get_direction(row, col))
            self.draw()
            if self.puzzle.is_solved():
                self.display_success_message()

    def get_direction(self, row, col):
        dx = row - self.puzzle.position[0]
        dy = col - self.puzzle.position[1]
        if dx == -1:
            return "up"
        elif dx == 1:
            return "down"
        elif dy == -1:
            return "left"
        elif dy == 1:
            return "right"

    def solve(self):
        self.puzzle = JigsawPuzzle(self.image, 3)
        self.draw()

    def display_success_message(self):
        success_window = tk.Toplevel(self.root)
        success_window.title("Congratulations!")
        success_label = tk.Label(success_window, text="Congratulations! You solved the puzzle!")
        success_label.pack()

    def start(self):
        self.draw()
        self.root.mainloop()

## 3. Running the Puzzle:

In [3]:
if __name__ == "__main__":
    root = tk.Tk()
    image_url = "https://t4.ftcdn.net/jpg/01/66/10/03/240_F_166100342_KbTGIRrnrlwGDZSXSMpH3zfn2dxyTKae.jpg"
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    puzzle_gui = PuzzleGUI(root, image, 3)
    puzzle_gui.start()
